# Recommending movies: retrieval
- https://www.tensorflow.org/recommenders/examples/basic_retrieval
    
Real-world recommender systems
1. retrieval: selecting an initial set of hundreds of candidates from all possible candidates
2. ranking: the outputs of the retrieval model -> the best possible handful of recommendations

Retrieval의 두 가지 sub-model
1. 쿼리 모델: the query representation (normally a fixed-dimensionality embedding vector)
2. 후보 모델: the candidate representation (an equally-sized vector)

쿼리모델, 후보모델 결과를 곱해서 -> 쿼리-후보간 선호도 점수 측정

implicit feedback vs. explicit feedback

In [1]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

You should consider upgrading via the '/Users/wonji/.pyenv/versions/3.7.7/bin/python3.7 -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/wonji/.pyenv/versions/3.7.7/bin/python3.7 -m pip install --upgrade pip' command.
ERROR: Could not find a version that satisfies the requirement scann
ERROR: No matching distribution found for scann
You should consider upgrading via the '/Users/wonji/.pyenv/versions/3.7.7/bin/python3.7 -m pip install --upgrade pip' command.


In [2]:
import os 
import pprint
import tempfile
from typing import Dict, Text
import numpy as np 
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

In [3]:
# Ratings data.
ratings = tfds.load("movielens/100k-ratings", split="train")
# Features of all the available movies.
movies = tfds.load("movielens/100k-movies", split="train")

In [4]:
for x in ratings.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'bucketized_user_age': 45.0,
 'movie_genres': array([7]),
 'movie_id': b'357',
 'movie_title': b"One Flew Over the Cuckoo's Nest (1975)",
 'raw_user_age': 46.0,
 'timestamp': 879024327,
 'user_gender': True,
 'user_id': b'138',
 'user_occupation_label': 4,
 'user_occupation_text': b'doctor',
 'user_rating': 4.0,
 'user_zip_code': b'53211'}


In [5]:
for x in movies.take(1).as_numpy_iterator():
    pprint.pprint(x)

{'movie_genres': array([4]),
 'movie_id': b'1681',
 'movie_title': b'You So Crazy (1994)'}


In [6]:
ratings = ratings.map(lambda x: {
    "movie_title": x["movie_title"],
    "user_id": x["user_id"],
})
movies = movies.map(lambda x: x["movie_title"])

In [7]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)
# train 8 test 2
train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

In [8]:
movie_titles = movies.batch(1_000)
user_ids = ratings.batch(1_000_000).map(lambda x: x["user_id"])

In [9]:
unique_movie_titles = np.unique(np.concatenate(list(movie_titles)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

### query model 

In [10]:
embedding_dimension = 32

In [11]:
# user_ids -> integer -> embedding
user_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_user_ids, mask_token=None),
  tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
])

### candidate model

In [12]:
movie_model = tf.keras.Sequential([
  tf.keras.layers.experimental.preprocessing.StringLookup(vocabulary=unique_movie_titles, mask_token=None),
  tf.keras.layers.Embedding(len(unique_movie_titles) + 1, embedding_dimension)
])

### metrics & loss

In [13]:
metrics = tfrs.metrics.FactorizedTopK(candidates=movies.batch(128).map(movie_model))
task = tfrs.tasks.Retrieval(metrics=metrics)

### The Full Model

In [14]:
class MovielensModel(tfrs.Model):
    def __init__(self, user_model, movie_model):
        super().__init__()
        self.movie_model: tf.keras.Model = movie_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task
            
    def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
        user_embeddings = self.user_model(features["user_id"])
        positive_movie_embeddings = self.movie_model(features["movie_title"])
        return self.task(user_embeddings, positive_movie_embeddings)
        # task = tfrs.tasks.Retrieval(metrics=metrics)

In [16]:
class NoBaseClassMovielensModel(tf.keras.Model):

    def __init__(self, user_model, movie_model):
        super().__init__()
        self.movie_model: tf.keras.Model = movie_model
        self.user_model: tf.keras.Model = user_model
        self.task: tf.keras.layers.Layer = task
            
    def train_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        with tf.GradientTape() as tape:
            # Loss computation.
            user_embeddings = self.user_model(features["user_id"])
            positive_movie_embeddings = self.movie_model(features["movie_title"])
            loss = self.task(user_embeddings, positive_movie_embeddings)
            
            regularization_loss = sum(self.losses)
            total_loss = loss + regularization_loss
        
        # train 단계에만 포함
        gradients = tape.gradient(total_loss, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradients, self.trainable_variables))
        
        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss
        
        return metrics

    def test_step(self, features: Dict[Text, tf.Tensor]) -> tf.Tensor:
        # Loss computation.
        user_embeddings = self.user_model(features["user_id"])
        positive_movie_embeddings = self.movie_model(features["movie_title"])
        loss = self.task(user_embeddings, positive_movie_embeddings)

        # Handle regularization losses as well.
        regularization_loss = sum(self.losses)

        total_loss = loss + regularization_loss
        metrics = {metric.name: metric.result() for metric in self.metrics}
        metrics["loss"] = loss
        metrics["regularization_loss"] = regularization_loss
        metrics["total_loss"] = total_loss

        return metrics

In [17]:
model = MovielensModel(user_model, movie_model)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [18]:
cached_train = train.shuffle(100_000).batch(8192).cache()
cached_test = test.batch(4096).cache()

In [20]:
model.fit(cached_train, epochs=3)
model.evaluate(cached_test, return_dict=True)

Epoch 1/3
10/10 [==============================] - 5s 498ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0038 - factorized_top_k/top_5_categorical_accuracy: 0.0261 - factorized_top_k/top_10_categorical_accuracy: 0.0509 - factorized_top_k/top_50_categorical_accuracy: 0.2025 - factorized_top_k/top_100_categorical_accuracy: 0.3340 - loss: 65609.7422 - regularization_loss: 0.0000e+00 - total_loss: 65609.7422
Epoch 2/3
10/10 [==============================] - 5s 480ms/step - factorized_top_k/top_1_categorical_accuracy: 0.0045 - factorized_top_k/top_5_categorical_accuracy: 0.0286 - factorized_top_k/top_10_categorical_accuracy: 0.0556 - factorized_top_k/top_50_categorical_accuracy: 0.2141 - factorized_top_k/top_100_categorical_accuracy: 0.3479 - loss: 65110.2727 - regularization_loss: 0.0000e+00 - total_loss: 65110.2727
Epoch 3/3
5/5 [==============================] - 2s 244ms/step - factorized_top_k/top_1_categorical_accuracy: 5.5000e-04 - factorized_top_k/top_5_categorical_accuracy

{'factorized_top_k/top_1_categorical_accuracy': 0.0005499999970197678,
 'factorized_top_k/top_5_categorical_accuracy': 0.00559999980032444,
 'factorized_top_k/top_10_categorical_accuracy': 0.0142000000923872,
 'factorized_top_k/top_50_categorical_accuracy': 0.10074999928474426,
 'factorized_top_k/top_100_categorical_accuracy': 0.20514999330043793,
 'loss': 28400.857421875,
 'regularization_loss': 0,
 'total_loss': 28400.857421875}